In [1]:
from parse_crystfel_stream import parse_crystfel_stream

stream_file = "/home/bubl3932/files/MIL101/xgandalf_iterations_max_radius_1.8_step_0.5/filtered_metrics/filtered_metrics.stream"

df_refl, df_cryst = parse_crystfel_stream(stream_file, debug=True)

Parsed 9864530 reflection rows total.
Parsed 6641 crystal rows (events).


In [2]:
from cctbx import crystal, miller

# Example placeholders (replace with your actual cell):
unit_cell = (87.4, 87.4, 87.4, 90.0, 90.0, 90.0)
space_group_symbol = "Fm-3m"  # or any other, e.g. "P212121"


In [3]:
import pandas as pd

# Suppose df_refl has columns ["h", "k", "l"] among others
all_indices = list(zip(df_refl["h"], df_refl["k"], df_refl["l"]))


In [5]:
from cctbx import crystal, miller
from cctbx.array_family import flex

# Example placeholders (replace with your actual cell):
unit_cell = (87.4, 87.4, 87.4, 90.0, 90.0, 90.0)
space_group_symbol = "Fm-3m"  # or any other, e.g. "Fm-3m"

cryst_sym = crystal.symmetry(
    unit_cell=unit_cell,
    space_group_symbol=space_group_symbol
)

# all_indices is your Python list of (h,k,l) tuples
# Convert to a flex.miller_index
all_indices_flex = flex.miller_index(all_indices)

ms = miller.set(
    crystal_symmetry=cryst_sym,
    indices=all_indices_flex,
    anomalous_flag=False  # or True as appropriate
)

ms_asu = ms.map_to_asu()  # now works
mapped_indices = ms_asu.indices()  # cctbx flex.miller_index

# Convert back to Python tuples if needed:
mapped_indices_python = list(mapped_indices)


In [6]:
# Convert (h,k,l) to tuple if needed:
mapped_indices_tuples = [tuple(idx) for idx in mapped_indices]

df_refl["asu_index"] = mapped_indices_tuples


In [7]:
grouped_by_asu = df_refl.groupby("asu_index")


In [8]:
counts = grouped_by_asu.size().reset_index(name="count")
counts.sort_values("count", ascending=False, inplace=True)
print(counts.head(10))


      asu_index  count
1636  (1, 7, 5)   2977
4867  (3, 7, 5)   2780
3265  (2, 6, 4)   2753
1639  (1, 9, 3)   2697
3269  (2, 8, 6)   2684
4870  (3, 9, 5)   2621
1635  (1, 7, 3)   2593
3268  (2, 8, 4)   2568
6427  (4, 8, 6)   2520
1641  (1, 9, 7)   2473


In [9]:
avg_intensity = grouped_by_asu["I"].mean().reset_index(name="I_mean")
print(avg_intensity.head(10))


   asu_index        I_mean
0  (0, 2, 2)  10123.251111
1  (0, 4, 0)  33906.740648
2  (0, 4, 2)   1531.951747
3  (0, 4, 4)  13793.428522
4  (0, 6, 0)    392.525243
5  (0, 6, 2)   2435.143500
6  (0, 6, 4)    373.087227
7  (0, 6, 6)   7584.626528
8  (0, 8, 0)   6207.925695
9  (0, 8, 2)    206.950762


In [10]:
for asu_idx, group_df in grouped_by_asu:
    print(f"Symmetry group: {asu_idx}, N reflections = {len(group_df)}")
    # group_df has all reflections that map to 'asu_idx'
    # Possibly do plotting or further analysis here...


Symmetry group: (0, 2, 2), N reflections = 27
Symmetry group: (0, 4, 0), N reflections = 108
Symmetry group: (0, 4, 2), N reflections = 332
Symmetry group: (0, 4, 4), N reflections = 433
Symmetry group: (0, 6, 0), N reflections = 267
Symmetry group: (0, 6, 2), N reflections = 1023
Symmetry group: (0, 6, 4), N reflections = 1309
Symmetry group: (0, 6, 6), N reflections = 1011
Symmetry group: (0, 8, 0), N reflections = 367
Symmetry group: (0, 8, 2), N reflections = 1339
Symmetry group: (0, 8, 4), N reflections = 1272
Symmetry group: (0, 8, 6), N reflections = 1325
Symmetry group: (0, 8, 8), N reflections = 880
Symmetry group: (0, 10, 0), N reflections = 332
Symmetry group: (0, 10, 2), N reflections = 1204
Symmetry group: (0, 10, 4), N reflections = 1155
Symmetry group: (0, 10, 6), N reflections = 1092
Symmetry group: (0, 10, 8), N reflections = 1144
Symmetry group: (0, 10, 10), N reflections = 732
Symmetry group: (0, 12, 0), N reflections = 294
Symmetry group: (0, 12, 2), N reflections =